In [ ]:
import pandas as pd
import numpy as np
from sklearn import ensemble,metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import datasets

## Data Preparation

In [ ]:
data=pd.read_csv('NIFTY.txt',header=None)
data.columns=['StockName','Date','Time','Open','Close','Low','High','Volume','Ext']
toData=data[['Open','Close','Low','Volume']]

In [ ]:
def preProcessDataScore(toData):
    toData['RollingOpen']=toData.rolling(window=5)[['Open']].mean()
    toData['RollingClose']=toData.rolling(window=5)[['Close']].mean()
    toData['RollingLow']=toData.rolling(window=5)[['Low']].mean()
    toData['StdDevOpen']=toData.rolling(window=5)[['Open']].std()
    toData['StdDevClose']=toData.rolling(window=5)[['Close']].std()
    toData['StdDevLow']=toData.rolling(window=5)[['Low']].std()
    toData=toData.fillna(method='bfill')
    return toData

In [ ]:
def preProcessDataTrain(toData):
    import pandas as pd
    data=pd.read_csv('../ZMOD/Data/NIFTY.txt',header=None)
    data.columns=['StockName','Date','Time','Open','Close','Low','High','Volume','Ext']
    toData=data[['Open','Close','Low','Volume']]
    toData['RollingOpen']=toData.rolling(window=5)[['Open']].mean()
    toData['RollingClose']=toData.rolling(window=5)[['Close']].mean()
    toData['RollingLow']=toData.rolling(window=5)[['Low']].mean()
    toData['StdDevOpen']=toData.rolling(window=5)[['Open']].std()
    toData['StdDevClose']=toData.rolling(window=5)[['Close']].std()
    toData['StdDevLow']=toData.rolling(window=5)[['Low']].std()
    toData=toData.fillna(method='bfill')
    return toData

In [ ]:
toData=preProcessDataScore(toData)
toData.head()

In [ ]:
toData.columns

In [ ]:
XVar=['Open', 'Close', 'Low',  'RollingOpen', 'RollingClose','RollingLow', 'StdDevOpen', 'StdDevClose', 'StdDevLow']
YVar=['Volume']

## PostProcess for 2nd Model

In [ ]:
# scored=model2.predict(toData[XVar])

In [ ]:
def preProcessData2nd(toData):#,scored):
#     toData['resi']=toData['Volume']-scored
#     toData['Rollingresi']=toData.rolling(window=5)[['resi']].mean()
#     toData['StdDevresi']=toData.rolling(window=5)[['resi']].std()
    toData=toData.fillna(method='bfill')
    toData['Volume']=toData['Volume'].apply(lambda x: 1 if x>90 else 0)
    return toData

In [ ]:
toData2=preProcessData2nd(toData)

In [ ]:
toData2.shape

In [ ]:
toData2.columns

In [ ]:
toData2.head()

# Dense Model

In [ ]:
verbose, epochs, batch_size = 1, 2, 32
#n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model2 = Sequential()
model2.add(Dense(32, input_dim=9))
# model.add(Dropout(0.5))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.fit(toData[XVar].values,toData[YVar].values, epochs=epochs, batch_size=32, verbose=verbose)

## 2nd Model LSTM

In [ ]:
kk=toData2[XVar].shape[0]
hh=toData2[XVar].shape[1]
XInde=toData2[XVar].values.reshape(kk,1,hh)
YDepe=toData2[YVar].values#.reshape(1,kk)

In [ ]:
XInde[0].shape

In [ ]:
verbose, epochs, batch_size = 1, 2, 32
#n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential()
model.add(LSTM(32, input_shape=(None,9)))
# model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(XInde,YDepe, epochs=epochs, batch_size=32, verbose=verbose)

In [ ]:
from nyoka.keras.keras_model_to_pmml import KerasToPmml
pmmlObj=KerasToPmml(model)
pmmlObj.export(open('LSTMModelTocheck.pmml','w'),0)

In [ ]:
pmmlObj2=KerasToPmml(model2)
pmmlObj2.export(open('DenseModelTocheck.pmml','w'),0)

In [ ]:
oo2=pmmlObj2.DeepNetwork[0].NetworkLayer[1]

In [ ]:
oo2.LayerWeights.valueOf_

In [1]:
import nyoka.PMML43Ext as ny
import sys
ny.parse('DenseModelTocheck.pmml',silence=True)

In [ ]:
oo=pmmlObj.DeepNetwork[0].NetworkLayer[1]#.LayerRecurrentWeights

In [ ]:
oo.LayerRecurrentWeights.valueOf_

In [ ]:
oo.export(sys.stdout,0)

In [2]:
ny.parse('LSTMModelTocheck.pmml',silence=True)

## Score a sample data

In [ ]:
dataTest=pd.read_csv('NIFTY.txt',header=None)
dataTest.columns=['StockName','Date','Time','Open','Close','Low','High','Volume','Ext']
dataTest=data[['Open','Close','Low','Volume']].head(10)

In [ ]:
dataTest=preProcessDataScore(dataTest)

In [ ]:
scoredTest=model1_.predict(dataTest[XVar])

In [ ]:
dataTest2=preProcessData2nd(dataTest,scoredTest)

In [ ]:
dataTest2.shape

In [ ]:
kk2=dataTest2[XVar2].shape[0]
hh2=dataTest2[XVar2].shape[1]
XInde2=dataTest2[XVar2].values.reshape(kk2,1,hh2)
YDepe2=dataTest2[YVar2].values#.reshape(1,kk)

In [ ]:
model.predict(XInde2).tolist()

# PostProcess for 2nd Model

In [ ]:
def postProcessing2(data,scored):
        import requests
        from requests.auth import HTTPBasicAuth
        import json
        import operator

        url = "https://ai.cumulocity.com/alarm/alarms"
        
        for i in range(scored):
            if lst[i] >= .6:
                defect_timestamp = data['timestamp'].values[i]

        headers = {
        'Content-Type': "application/vnd.com.nsn.cumulocity.alarm+json",
        'Accept': "application/vnd.com.nsn.cumulocity.alarm+json",
        }

        payload_Critical_Drink = {
            "source": {
                "id": "117701" },
            "type": "Anamoly",
            "text": "There has been an Anamoly and it occured in this timestamp:"+ str(defect_timestamp),
            "severity": "CRITICAL",
            "status": "ACTIVE",
            "time": "2014-03-03T12:03:27.845Z"
        }


        print ('File read')

        auth = HTTPBasicAuth('Rainer.Burkhardt@softwareag.com', 'cum 2418 Point Loma')

        if pred == 1:
            response = requests.request("POST", url, data=json.dumps(payload_Critical_Drink),headers=headers,auth=auth)

        print ('Done')

# PMML Conversion

In [ ]:
from nyoka import model_to_pmml

In [ ]:
toExportDict={
    'model1':{
        'hyperparameters':None,
        'preProcessingScript':{'scripts':[preProcessDataTrain,preProcessDataScore], 'scriptpurpose':['train','score']},
        'pipelineObj':None,
        'modelObj':model1_,
        'featuresUsed':XVar,
        'targetName':'Volume',
        'postProcessingScript':None,
        'taskType': 'trainAndscore'
    },
    'model2':{
    'hyperparameters':param,
    'preProcessingScript':{'scripts':[preProcessData2nd], 'scriptpurpose':['score']},
    'pipelineObj':None,
    'modelObj':model,
    'featuresUsed':XVar2,
    'targetName':'Volume',
    'postProcessingScript':{'scripts':[postProcessing2], 'scriptpurpose':['score']},
    'taskType': 'score'
    }
}

pmml = model_to_pmml(toExportDict, pmml_f_name="UseCase1.pmml")